## Hands-On ML with Scikit-Learn

Taken from Geron, A. (2023). Hands-On Machine Learning with Scikit-Learn, Keras and TensorFlow. O'Reilly: 3rd edition.

### Chapter 2. End-to-End Machine Learning Project

1. Look at the big picture

2. Get the data

3. Explore and visualize the data to gain insights

4. Prepare the data for ML algorithms

5. Select a model and train it

6. Fine-tune your model

7. Present your solution

8. Launch, monitor and maintain your system


**Working with real data**:

* [OpenML.org](https://openml.org/)

* [Kaggle](https://www.kaggle.com/datasets)

* [PapersWithCode.com](https://paperswithcode.com/datasets)

* [UC Irvine ML repo](https://archive.ics.uci.edu/)

* [Amazon AWS datasets](https://registry.opendata.aws/)

* [TensorFlow datasets](https://www.tensorflow.org/datasets)

* [DataPortals (Meta-portal)](https://dataportals.org/)

* [Open Data Monitor (Meta-portal)](https://opendatamonitor.eu/frontend/web/index.php?r=dashboard%2Findex)

### The Project: Housing Prices in California

#### 1. Look at the big picture

Use California census data to **build a model of housing prices in the state**. 

This data includes metrics such as the population, median income, and median housing price for each *block group* ('district') in California.

Your model **should learn from this data and be able to predict the median housing price** in any district, given all the other metrics.

##### Frame the Problem

* *Know the objective*:

The first question you and your customer should ask is: *How does the company expect to use and benefit from this model?*

This will help you frame the problem, select algorithms and performance measures and how much effort you will spend teaking it.

Your customer answers that your model's output (a prediction of a district's median housing price) will be fed to another ML system (see Figure 2-2). This downstream system will determin whether it is worth investing in a given area.

![](https://abhijitramesh.me/static/images/part2-learning-hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/Screenshot_2021-03-05_at_1.17.49_PM.png)

* *Know the current situation*:

The next question is: *What does the current situation look like?*

This will give you a reference for performance, as well as insights on how to solve the problem.

Your customer answers that the district housing prices are currently estimated manually by experts: a team gathers up-to-date information about a district and they estimate it using complex rules.

* *Design your solution*

Finally, ask: *What kind of training supervision the model will need? Is it a classification, regression or reinforcement learning task? Should I use batch learning or online learning techniques?...*

The solution required in this case is a typical **regression task**, since the model is asked to predict a value:

* Multiple regression: multiple features are required to make a prediction

* Univariate regression: model only predicts a single value for each district

* Plain batch learning: there is no continuous flow of data coming into the system and no particular need to adjust to changing data rapidly. Data set is also small enough to fit in memory.

##### Pipelines

A *Pipeline* is a sequence of data processing components. Pipelines are very common in ML systems, since there is a lot of data to manipulate and many data transformations to apply.

Components typically run asynchronously.

* Each component pulls in a large amount of data, processes it and spits out the result in another data store.

* Then, some time later, the next component in the pipeline pulls in this data and spits out its own output.

* Each component is fairly self-contained: the interface between components is simply the data store.

##### Select Performance Measures

A typical performance measure for regression problems is the **Root Mean Square Error (RMSE)**. 

$$
RMSE(\mathbf{X}, h) = \sqrt{\frac{1}{m}\sum_{i=1}^m\big(h(\mathbf{x}^{(i)}-y^{(i)}\big)^2}
$$

where,

$m$ = the number of instances in the dataset. For example, if you are evaluating the RMSE on a validation set of 2000 districts, then $m=2,000$

$\mathbf{x}^{(i)}$ = a vector of all the feature values (excluding the label) of the $i$-th instance in the dataset.

$y^{(i)}$ = the label or desired output value for that instance.

* For example, if the 1st district in the dataset is located at longitude $-118.29^{\circ}$, latitude $33.91^{\circ}$, and it has $1,416$ inhabitants with a median income of $\$38,372$, and the median house value is $\$156,400$ (ignoring other features for now). Then,

$$
\mathbf{x}^{(1)} = 
\begin{pmatrix}
-118.29\\
33.91\\
1,416\\
38,372
\end{pmatrix}
$$

and 

$$
y^{(1)} = 156,400
$$

$\mathbf{X}$ = a matrix containing all the feature values (excluding labels) of all instances in the dataset. There is one row per instance, and the $i$-th row is equal to the *transpose* of $\mathbf{x}^{(i)}$, noted as $(\mathbf{x}^{(i)})^\top$

* For example, if the first district is as just described, then the matrix $\mathbf{X}$ looks like this:

$$
\mathbf{X} = 
\begin{pmatrix}
(\mathbf{x}^{(1)})^\top\\
(\mathbf{x}^{(2)})^\top\\
\vdots\\
(\mathbf{x}^{(m-1)})^\top\\
(\mathbf{x}^{(m)})^\top\\
\end{pmatrix}
=
\begin{pmatrix}
-118.29 & 33.91 & 1,416 & 38,372\\
\vdots & \vdots & \vdots & \vdots
\end{pmatrix}
$$

$h$ = your system's prediction function: *hypothesis*. That is, the predicted value of your system: $\hat{y}^{(i)}=h(\mathbf{x}^{(i)})$.

* For example, if your system predicts that the median housing price in the first district is $\$158,400$, then $\hat{y}^{(1)}=158,400$. The prediction error for this district is $\hat{y}^{(1)} - y^{(1)} = 2,000$

Another performance measure used is the **Mean Absolute Error (MAE)** or average absolute deviation:

$$
MAE(\mathbf{X},h) = \frac{1}{m}\sum_{i=1}^m \big|h(\mathbf{x}^{(i)}-y^{(i)})\big|
$$

Both the RMSE and the MAE are ways to measure *the distance between two vectors*: the vector of predictions and the vector of target values.

Various distance measures or *norms* are possible:

* **Euclidean norm**: Computing the root of a sum of squares, like in RSME. It is also called the $\ell_2$ norm and noted using $\parallel\cdot\parallel$

* **Manhattan norm**: Computing the sum of absolutes, like in MAE. It is also called the $\ell_1$ norm. 